# Transformers pour la classification de texte

L'objectif de ce TP est d'implémenter une version simplifiée d'un Transformer pour résoudre un problème de classification de texte.

Nous utiliserons comme exemple illustratif une base de données présente dans la librairie ```Keras``` consistant en des critiques de films postées sur le site IMDB, accompagnées d'une note qui a été binarisée pour révéler le caracète positif, ou négatif, de la critique.

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2024-09-20 10:09:27.027372: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-20 10:09:27.059539: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-20 10:09:27.733704: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Implémentation d'un bloc de base de Transformer


<center><img src="https://drive.google.com/uc?id=1w1CyLROPq-EWMd-Spr6wR596QEx1KpNa"> </center>
<caption><center> Figure 1: Schéma de l'architecture de GPT 1</center></caption>

La figure ci-dessus présente l'architecture de GPT-1. Le bloc de base d'un Transformer est composé d'un bloc de *Self-Attention*, d'une couche de ```Layer Normalization``` (similaire à la ```Batch Normalization```), d'une couche dense et enfin d'une nouvelle couche de ```Layer Normalization```.

Pour implémenter la *Self-Attention*, vous pouvez utiliser la fonction ```layers.MultiHeadAttention``` (à vous de regarder quels en sont les paramètres dans la documentation).

**ATTENTION: Pour implémenter un Transformer de type GPT, la couche doit masquer l'attention portée par un token aux tokens suivants !**

**Rappel**: Une couche d'Attention *Multi-Head*  se présente sous la forme ci-dessous à gauche, avec le mécanisme d'attention détaillé à droite :


<center>

<img src="https://drive.google.com/uc?id=1UTozEHtsZ3xy61XJqn_Eug-7mn7bFp9m">
<img src="https://drive.google.com/uc?id=1aTttpp1OOasVVZAi3lWwosh68VnBjQnz">
</center>

D'après vous, combien de paramètres comporte une couche d'attention à 2 têtes, pour un *Embedding* de dimension 32 ?

pour 1 tête : 3 couches de 32*32 : 3*(32*32 + 32) poid synaptiques + biais

pour 2 tete on double puis 64*32 + 32 pour la sortie concat 2*32 avec la sortie 32 plus 32 biais

In [22]:
class TransformerBlock(layers.Layer):
    # embed_dim désigne la dimension des embeddings maintenus à travers les différentes couches,
    # et num_heads le nombre de têtes de la couche d'attention.
    def __init__(self, embed_dim, num_heads):
        super().__init__()
        # Définition des différentes couches qui composent le bloc
        # Couche d'attention
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        # Première couche de Layer Normalization
        self.layernorm1 = layers.LayerNormalization()
        # Couche Dense (Feed-Forward)
        self.ffn = layers.Dense(embed_dim, activation='relu')
        # Deuxième couche de normalisation
        self.layernorm2 = layers.LayerNormalization()

    def call(self, inputs):
        # Application des couches successives aux entrées
        attention = self.att(inputs, inputs)
        connexion = self.layernorm1(attention + inputs)
        ff = self.ffn(connexion)
        return self.layernorm2(ff + connexion)

## Implémentation de la double couche d'Embedding

La séquence d'entrée est convertie en *Embedding* de dimension ```embed_dim```.
L'*Embedding* final est constitué de la somme de deux *Embedding*, le premier encodant un mot, et le second encodant la position du mot dans la séquence.

La couche d'*Embedding* de Keras (```layers.Embedding```) est une sorte de table associant à un indice en entrée un vecteur de dimension ```embed_dim```. Chaque coefficient de cette table est en fait un paramètre apprenable.

D'après vous combien de paramètres contiendrait une couche d'*Embedding* associant un vecteur de dimension 32 à chacun des 20000 mots les plus courants du vocabulaire extrait de la base de données que nous allons utiliser ?
Et combien pour l'*Embedding* qui associe un vecteur de dimension 32 à chaque position d'un séquence de longueur ```maxlen``` 

20000*32

32*maxlen

In [29]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        # Définition des différentes couches qui composent le bloc Embedding
        # Embedding de mot
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        # Embedding de position
        self.pos_emb =layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        # Calcul de l'embedding à partir de l'entrée x
        # ATTENTION : UTILISER UNIQUEMENT DES FONCTIONS TF POUR CETTE PARTIE
        # Récupération de la longueur de la séquence
        maxlen = tf.shape(x)[-1]
        # Création d'un vecteur [0, 1, ..., maxlen] des positions associées aux
        # mots de la séquence
        positions = tf.range(start=0, limit=maxlen, delta=1)
        # Calcul des embeddings de position
        positions_emb = self.pos_emb(positions)
        # Calcul des embeddings de mot
        words_emb = self.token_emb(x)
        return words_emb + positions_emb

## Préparation de la base de données

In [27]:
# Taille du vocabulaire considéré (on ne conserve que les 20000 mots les plus courants)
vocab_size = 20000
# Taille maximale de la séquence considérée (on ne conserve que les 200 premiers mots de chaque commentaire)
maxlen = 200

# Chargement des données de la base IMDB
(x_train, y_train), (x_val, y_val) = keras.datasets.imdb.load_data(num_words=vocab_size)

print(len(x_train), "séquences d'apprentissage")
print(len(x_val), "séquences de validation")

# Padding des séquences : ajout de "0" pour compléter les séquences trop courtes
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.preprocessing.sequence.pad_sequences(x_val, maxlen=maxlen)

25000 séquences d'apprentissage
25000 séquences de validation


## Création du modèle

Pour assembler le modèle final, il faut, partant d'une séquence de longueur ```maxlen```, calculer les Embedding puis les fournir en entrée d'une série de blocs Transformer. Pour ce TP, **commencez par ne mettre qu'un seul bloc Transformer**. Vous pourrez en ajouter plus tard si vous le souhaitez.

Pour construire la tête de projection du réseau, vous pouvez moyenner les activations en sortie du bloc Transformer par élément de la séquence grâce à un *Global Average Pooling* (1D !), à relier à une couche dense (par exemple comportant 20 neurones) et enfin à la couche de sortie du réseau.

In [38]:
embed_dim = 32  # Dimension de l'embedding pour chaque mot
num_heads = 2  # Nombre de têtes d'attention

# A COMPLETER
inputs = layers.Input(shape=(maxlen, ))
x = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)(inputs)
x = TransformerBlock(embed_dim, num_heads)(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dense(20, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_14 (InputLayer)     │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding_10 │ (None, 200, 32)        │       646,400 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block_14            │ (None, 200, 32)        │         9,600 │
│ (TransformerBlock)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d_6      │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 20)             │           660 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1)              │            21 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 656,681 (2.51 MB)

 Trainable params: 656,681 (2.51 MB)

 Non-trainable params: 0 (0.00 B)

Enfin vous pouvez lancer l'apprentissage, avec par exemple l'optimiseur Adam. Inutile de lancer de trop nombreuses *epochs*, le réseau sur-apprend très vite !

In [39]:
# A COMPLETER
model.compile(
    optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    x_train, y_train, batch_size=32, epochs=5, validation_data=(x_val, y_val)
)

Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 29s 35ms/step - accuracy: 0.7474 - loss: 0.4877 - val_accuracy: 0.8825 - val_loss: 0.2847
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 28s 35ms/step - accuracy: 0.9335 - loss: 0.1816 - val_accuracy: 0.8684 - val_loss: 0.3222
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 27s 35ms/step - accuracy: 0.9652 - loss: 0.1071 - val_accuracy: 0.8592 - val_loss: 0.3717
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 28s 36ms/step - accuracy: 0.9806 - loss: 0.0645 - val_accuracy: 0.8526 - val_loss: 0.5004
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 28s 35ms/step - accuracy: 0.9880 - loss: 0.0419 - val_accuracy: 0.8452 - val_loss: 0.6126


**Questions subsidiaires**:



1.   Comparez les résultats à ceux d'un LSTM bi-directionnel.
2.   **Plus dur**: GPT bénéficie d'un pré-entraînement non supervisé sur des données issues de gigantesques bases de texte. L'idée, résumée dans l'extrait de l'article copié ci-dessous, consiste à pré-entraîner le modèle à prédire le prochain mot d'une séquence fournie. De larges bases de données, comme [WikiText](https://huggingface.co/datasets/wikitext), permettent de pré-entraîner efficacement le réseau, particulièrement dans notre cas la couche d'*Embedding* qui contient la majorité des paramètres du réseau.

<center><img src="https://drive.google.com/uc?id=1RWPVSAEA5frRvqHkOxw6h1MDRe-fT0sC"> </center>


## Quelques éléments pour aller plus loin

Chargement de la base de données WikiText

In [ ]:
!pip install datasets

from datasets import load_dataset

dataset = load_dataset("wikitext", "wikitext-103-v1")

print(dataset)

La difficulté est maintenant de travailler cette base de données pour produire des séquences, en réutilisant les mêmes numéros de tokens de la base IMDB utilisée précédemment...